In [1]:
from pylab import imread,subplot,imshow,title,gray,figure,show,NullLocator
from PIL import Image
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt 

In [2]:
x_file = open("sample_train_dataset.txt", "r")
a = []
a_name = []
name_classes = []
for file in x_file:
    sp = file.split() 
    arr = np.array(Image.open(sp[0]).convert("L").resize((32,32))).flatten()
    a.append(arr)
    a_name.append(str(sp[1]))
    name_classes.append(str(sp[1]))

In [3]:
def PCA(a):
    a = np.array(a)
    mean1 = a - np.mean(a,axis=0)
    v, eigenvalues , eigenvectors = np.linalg.svd(mean1)
    coefficiets = np.matmul(mean1 , eigenvectors.T) 
    arr1 = np.array(coefficiets[:,:32])
    b = np.ones((len(arr1),1))
    arr1 = np.hstack((arr1,b))
    return np.array(arr1) , eigenvectors[0:32,:]

In [4]:
new_img,eigenvectors = PCA(a)
print(new_img.shape)
a_name = np.array(a_name)

(400, 33)


In [5]:
unique_classes = list(set(a_name))
assign_int = {}
assign_name = {}
for i in range(len(unique_classes)):
    assign_int[i] = unique_classes[i]
    assign_name[unique_classes[i]] = i
for i in range(len(a_name)):
    a_name[i] = assign_name[a_name[i]]
a_name = a_name.astype(int)

In [6]:
print(assign_int)
print(a_name)
print(unique_classes)
print(assign_name)

{0: '5', 1: '1', 2: '4', 3: '2', 4: '3', 5: '7', 6: '6', 7: '0'}
[0 0 6 1 1 4 4 6 4 7 6 5 7 6 0 0 4 1 6 0 0 7 1 6 2 5 7 3 7 2 5 0 2 0 1 0 0
 7 6 4 1 7 0 6 0 5 3 3 7 2 6 3 1 0 6 5 7 6 0 5 0 4 7 4 4 3 1 4 4 0 1 0 4 3
 2 7 7 1 2 4 2 5 2 7 5 5 3 0 4 6 5 5 0 2 4 7 3 0 5 7 3 4 0 4 2 4 7 0 0 7 2
 7 0 0 5 2 4 6 2 1 7 3 5 4 0 6 1 7 2 3 4 5 2 7 2 2 2 7 1 0 3 7 6 1 7 7 6 3
 5 6 5 2 5 3 7 1 0 0 1 4 1 5 4 0 1 6 6 4 5 3 0 6 5 0 3 3 0 5 6 4 1 1 2 6 0
 7 6 6 3 5 3 6 4 1 1 6 3 1 1 3 6 2 5 6 3 0 6 5 0 6 5 3 1 7 6 1 5 3 6 2 4 5
 5 1 2 2 6 4 2 3 0 3 0 7 0 2 6 7 0 1 1 6 2 1 5 7 6 3 5 6 1 5 5 2 2 4 0 4 7
 2 6 7 7 6 2 4 3 2 4 3 3 6 4 7 7 6 3 3 4 7 2 5 2 2 6 6 5 0 6 7 2 2 6 7 3 0
 0 1 5 7 4 4 2 4 2 5 4 4 5 4 1 4 3 1 4 7 2 3 3 4 2 0 3 0 4 6 5 2 1 2 5 4 3
 3 5 4 7 3 2 6 1 5 7 2 6 3 3 3 3 3 1 6 5 2 6 7 1 7 7 5 4 7 5 3 3 0 5 0 5 1
 6 3 0 0 4 4 5 1 1 1 1 3 5 6 2 3 6 2 1 5 3 4 2 6 7 1 2 0 7 2]
['5', '1', '4', '2', '3', '7', '6', '0']
{'7': 5, '5': 0, '1': 1, '4': 2, '2': 3, '6': 6, '3': 4, '0': 7}


In [7]:
num_classes = len(set(a_name))
X = np.array(new_img.T)
D = len(X)
W = np.random.randn(num_classes, D) * 0.001
lr = 1e-4
reg = 500
num_iters=10000
Y = np.array(a_name)
print(X.shape,Y.shape,W.shape)

(33, 400) (400,) (8, 33)


In [8]:
def loss_grad(W, X, y, reg):
    loss = 0 
    grad = np.zeros_like(W)
    dim, num_train = X.shape
    scores = W.dot(X)
    #print(W)
    #print(scores)
    scores -= np.max(scores)
    scores_exp = np.exp(scores)
    scores_exp = scores_exp*1e50
#     print(num_train)
    correct_scores_exp = scores_exp[y, range(num_train)]
#     print(correct_scores_exp.shape,scores_exp.shape)
    scores_exp_sum = np.sum(scores_exp, axis=0)
    #print(scores_exp_sum)
    loss = -np.sum(np.log(correct_scores_exp / scores_exp_sum))
    loss /= num_train
    #loss += 0.5 * reg * np.sum(W * W)
    scores_exp_normalized = scores_exp / scores_exp_sum
    scores_exp_normalized[y, range(num_train)] -= 1
    grad = scores_exp_normalized.dot(X.T)
    grad /= num_train
    #grad += reg * W
    
    return loss, grad

In [9]:
losses_history = []
for i in range(num_iters):
    loss, grad =loss_grad(W, X, Y, reg)
    #print(loss)
    losses_history.append(loss)
    W -= lr * grad
    if(loss<1):
        break;
    #if (i % 100 == 0):
    #print('iteration %d/%d: loss %f ' % (i, num_iters, loss))


In [15]:
 def predict(X):
    pred_ys = np.zeros(X.shape[1])
    scores = W.dot(X)
    print(scores.shape,X.shape)
    pred_ys = np.argmax(scores, axis=0)
    return pred_ys


In [16]:
x_files = open("sample_test_dataset.txt", "r")
res = []
file_name = []
for file in x_files:
    file = file.split('\n')
    f = Image.open(str(file[0])).convert("L").resize((32,32))
    file_name.append(str(file[0]))
    arr = np.asarray(f)
    arr = arr.reshape(-1)
    res.append(arr)
    f.close
    
print(np.array(res).shape)
print(np.mean(a,axis=0).shape)

(120, 1024)
(1024,)


In [17]:
res = res - np.mean(a,axis=0)
coefficiets = np.matmul(res , eigenvectors.T)
b = np.ones((len(res),1))
res = np.hstack((coefficiets,b))
pred = predict(np.array(res).T)
np.array(res).T.shape

(8, 120) (33, 120)


(33, 120)

In [19]:
pred

array([2, 2, 0, 5, 1, 7, 6, 2, 1, 2, 4, 1, 7, 2, 0, 5, 1, 2, 0, 0, 0, 6,
       3, 3, 0, 0, 5, 7, 6, 6, 0, 7, 2, 1, 1, 3, 1, 5, 2, 1, 0, 2, 5, 7,
       3, 3, 3, 4, 3, 4, 5, 7, 7, 3, 6, 2, 6, 3, 3, 5, 7, 7, 5, 2, 4, 6,
       5, 1, 6, 2, 4, 4, 3, 0, 6, 7, 4, 5, 7, 6, 6, 7, 6, 0, 5, 6, 4, 0,
       3, 4, 4, 2, 6, 1, 6, 5, 7, 0, 4, 1, 5, 4, 7, 7, 2, 6, 1, 4, 3, 6,
       0, 7, 4, 3, 6, 5, 3, 6, 4, 2])

In [14]:
for i in range(len(pred)):
     if (assign_int[pred[i]] == file_name[i][17]):
            acc=acc+1
print(acc)
        

0
